## Importação de bibliotecas e Tratamento dos dados

In [55]:
#importar bibliotecas importantes 
import pandas as pd
import sqlite3
import csv

In [2]:
# Listas de colunas a serem mantidas em cada DataFrame
colunas_perfil = ["SG_UF",'NM_MUNICIPIO', "CD_MUNICIPIO", "DS_FAIXA_ETARIA", "DS_GENERO", "DS_GRAU_ESCOLARIDADE", "QT_ELEITORES_PERFIL"]
colunas_resultados = ["SG_UF", "CD_MUNICIPIO", "NM_MUNICIPIO", "NM_VOTAVEL", "QT_VOTOS"]

# Ler os dados dos arquivos CSV para DataFrames
perfil_eleitorado = pd.read_csv('perfil_eleitorado_2020.csv', sep=';', encoding='Latin 1')
resultados = pd.read_csv("SP_turno_1.csv", sep=';', encoding='Latin 1')

# Manter apenas as colunas necessárias
perfil_eleitorado = perfil_eleitorado[colunas_perfil]
resultados = resultados[colunas_resultados]

# Realizar o tratamento dos dados
# Substituir campos vazios por NaN e -1 por NaN nos campos numéricos
perfil_eleitorado.replace({'#NULO#': None, -1: None}, inplace=True)
resultados.replace({'#NULO#': None, -1: None}, inplace=True)

# Remover registros com valores nulos dos DataFrames
perfil_eleitorado.dropna(inplace=True)
resultados.dropna(inplace=True)

# Salvar DataFrames como arquivos CSV, substituindo se já existirem
perfil_eleitorado.to_csv('perfil_eleitorado_cleaned.csv', index=False, sep=';', encoding='Latin 1', mode='w')
resultados.to_csv('resultados_cleaned.csv', index=False, sep=';', encoding='Latin 1', mode='w')

print("DataFrames salvos como arquivos CSV.")

DataFrames salvos como arquivos CSV.


## Utilizar SQLite para armazenar os dados e realizar consultas

In [3]:
# Conectar ao banco de dados
conn = sqlite3.connect('eleicoes.db')
cursor = conn.cursor()

# Criar tabelas
create_table_perfil = '''
CREATE TABLE IF NOT EXISTS perfil_eleitorado (
    SG_UF TEXT,
    NM_MUNICIPIO TEXT,
    CD_MUNICIPIO INTEGER,
    DS_FAIXA_ETARIA TEXT,
    DS_GENERO TEXT,
    DS_GRAU_ESCOLARIDADE TEXT,
    QT_ELEITORES_PERFIL INTEGER
);
'''

create_table_resultados = '''
CREATE TABLE IF NOT EXISTS resultados (
    SG_UF TEXT,
    CD_MUNICIPIO INTEGER,
    NM_MUNICIPIO TEXT,
    NM_VOTAVEL TEXT,
    QT_VOTOS INTEGER
);
'''

cursor.execute(create_table_perfil)
cursor.execute(create_table_resultados)

# Importar dados do arquivo CSV para a tabela perfil_eleitorado
with open('perfil_eleitorado_cleaned.csv', 'r', newline='', encoding='latin1') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter=';')
    for row in csvreader:
        values = [row[col] for col in colunas_perfil]
        cursor.execute("INSERT INTO perfil_eleitorado VALUES (?, ?, ?, ?, ?, ?, ?)", values)

# Importar dados do arquivo CSV para a tabela resultados
with open('resultados_cleaned.csv', 'r', newline='', encoding='latin1') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter=';')
    for row in csvreader:
        values = [row[col] for col in colunas_resultados]
        cursor.execute("INSERT INTO resultados VALUES (?, ?, ?, ?, ?)", values)

# Commit as alterações e fechar a conexão
conn.commit()
conn.close()

## Consulta 1: Top 10 candidatos mais votados de acordo com o município específico 

In [16]:
# Conectar ao banco de dados
conn = sqlite3.connect('eleicoes.db')
cursor = conn.cursor()

# Candidato a ser especificado
municipio_especifico = input('Município desejado: ').upper()

# Consulta SQL para encontrar o candidato mais votado em cada município(top 10)
query = '''
SELECT
    NM_VOTAVEL,sum(QT_VOTOS) AS TOTAL_VOTOS
FROM
    resultados r
WHERE
    r.NM_MUNICIPIO = ? AND
    NM_VOTAVEL != 'Nulo' AND -- remover votos nulos
    NM_VOTAVEL != 'Branco' -- remover votos em branco
    GROUP BY r.NM_VOTAVEL
ORDER BY TOTAL_VOTOS DESC
LIMIT 10;
'''

# Executar a consulta
cursor.execute(query,(municipio_especifico,))

# Obter os resultados
results = cursor.fetchall()

# Exibir os resultados
print(f'Os candidatos mais votados de {municipio_especifico} em ordem decrescente :')
print('----------------------------------------------------')
for row in results:
    print(f'Candidato:{row[0]}, Votos Totais: {row[1]}')
   
    

# Fechar a conexão
conn.close()


Município desejado: ITAPETININGA
Os candidatos mais votados de ITAPETININGA em ordem decrescente :
----------------------------------------------------
Candidato:SIMONE MARQUETTO, Votos Totais: 46913
Candidato:MILTON NERY, Votos Totais: 15466
Candidato:RICARDO BARBARA, Votos Totais: 10996
Candidato:MIGUEL TURMEIRO, Votos Totais: 1854
Candidato:MDB, Votos Totais: 1817
Candidato:FUAD, Votos Totais: 1542
Candidato:UAN MOREIRA, Votos Totais: 1533
Candidato:GÊ DE ARAÚJO, Votos Totais: 1287
Candidato:ZE DA ESCOLA, Votos Totais: 1162
Candidato:ETSON BRUN, Votos Totais: 1147


## Consulta 2: Encontrar o munícipio aonde o candidato específico foi mais votado 

In [60]:
# Conectar ao banco de dados
conn = sqlite3.connect('eleicoes.db')
cursor = conn.cursor()

# Candidato a ser especificado
candidato_especifico = 'simone marquetto'.upper()

# Consulta SQL para encontrar o candidato mais votado em cada município
query = '''
SELECT
    r.NM_MUNICIPIO,
    SUM(r.QT_VOTOS) AS MaxVotos
FROM
    resultados r
WHERE
    r.NM_VOTAVEL = ?
GROUP BY
    r.NM_MUNICIPIO;

'''

# Executar a consulta
cursor.execute(query,(candidato_especifico,))

# Obter os resultados
results = cursor.fetchall()

# Exibir os resultados
for row in results:
    print("Município:",row[0])
    print("Total de Votos:",row[1])

# Fechar a conexão
conn.close()


Município: ITAPETININGA
Total de Votos: 46913


## Consulta 3: Perfil do eleitor de acordo com o município específico

In [13]:
import sqlite3

# Conectar ao banco de dados
conn = sqlite3.connect('eleicoes.db')
cursor = conn.cursor()

# Municipio a ser especificado
municipio_especifico = input('Municipio desejado: ').upper()

# Consulta SQL para obter os valores mais frequentes nas colunas DS_GENERO, DS_FAIXA_ETARIA e DS_GRAU_ESCOLARIDADE ordenados por numero total de eleitores com perfil
query = f'''
SELECT
    NM_MUNICIPIO,
    (SELECT DS_GENERO FROM perfil_eleitorado WHERE SG_UF = 'SP' AND NM_MUNICIPIO = ? GROUP BY DS_GENERO ORDER BY SUM(QT_ELEITORES_PERFIL) DESC LIMIT 1) AS Genero_Mais_Frequente,
    (SELECT DS_FAIXA_ETARIA FROM perfil_eleitorado WHERE SG_UF = 'SP' AND NM_MUNICIPIO = ? GROUP BY DS_FAIXA_ETARIA ORDER BY SUM(QT_ELEITORES_PERFIL) DESC LIMIT 1) AS Faixa_Etaria_Mais_Frequente,
    (SELECT DS_GRAU_ESCOLARIDADE FROM perfil_eleitorado WHERE SG_UF = 'SP' AND NM_MUNICIPIO = ? GROUP BY DS_GRAU_ESCOLARIDADE ORDER BY SUM(QT_ELEITORES_PERFIL) DESC LIMIT 1) AS Grau_Escolaridade_Mais_Frequente,
     SUM(QT_ELEITORES_PERFIL) AS VOTOS_TOTAL
FROM
    perfil_eleitorado
WHERE
    SG_UF = 'SP' AND
    NM_MUNICIPIO = ? 
GROUP BY
    NM_MUNICIPIO;
'''

# Executar a consulta
cursor.execute(query, (municipio_especifico, municipio_especifico, municipio_especifico, municipio_especifico))

# Obter os resultados
results = cursor.fetchall()

# Exibir os resultados
for row in results:
    print("Município:", row[0])
    print("Gênero Mais Frequente:", row[1])
    print("Faixa Etária Mais Frequente:", row[2])
    print("Grau de Escolaridade Mais Frequente:", row[3])
    print("Total de eleitores:", row[4])

# Fechar a conexão
conn.close()


Municipio desejado: sorocaba
Município: SOROCABA
Gênero Mais Frequente: FEMININO
Faixa Etária Mais Frequente: 35 a 39 anos                  
Grau de Escolaridade Mais Frequente: ENSINO MÉDIO COMPLETO
Total de eleitores: 485962
